In [20]:
import numpy as np
import pandas as pd
import cv2
import os

image3 = cv2.imread("data/memorial.hdr",cv2.IMREAD_ANYDEPTH)


In [21]:
gray_image3 = np.mean(image3,axis=2)
############*Log image***********8#
log_image = np.log(gray_image3)
np.min(log_image)
log_image_t = (255/(np.max(log_image)-np.min(log_image)))*log_image -(255/(np.max(log_image)-np.min(log_image)))*np.min(log_image)


In [22]:
######Filters 
def gaussian_kernel_1d(size:int,sigma:float):
    distance = np.square(np.linspace(-(size//2),size//2,size))
    kernel = np.exp(-0.5 * distance / np.square(sigma) )
    return kernel/np.sum(kernel)